In [2]:
###CONFIGURANDO TABELAS###

import pandas as pd
import holidays
from datetime import datetime, timedelta


# Caminho para salvar o arquivo no Google Drive
file_path = r'C:\Users\henri\OneDrive\Desktop\DIVERSOS\OLIST/'

In [3]:

###CARREGANDO ARQUIVOS###

datas = pd.read_csv(file_path+'bronze/dim_data.csv')
pedidos = pd.read_csv(file_path+'bronze/pedidos.csv')
produtos = pd.read_csv(file_path+'bronze/produtos.csv')
localizacoes = pd.read_csv(file_path+'bronze/localizacoes.csv')

localizacoes = localizacoes.drop('Unnamed: 0', axis = 1)
produtos = produtos.drop('Unnamed: 0', axis = 1)

datas.to_csv(file_path+'silver/dim_data.csv')
localizacoes.to_csv(file_path+'silver/dim_localizacoes.csv')

In [4]:

###CRIANDO PRODUTOS###

produtos['volume'] = produtos['comprimento_cm'] * produtos['altura_cm'] * produtos['largura_cm']

produtos['volume'] = produtos['volume']/(100*100*100)
produtos['massa'] = produtos['peso_g']/1000

produtos.rename(columns = {'id_produto':'idproduto'}, inplace = True)

produtos = produtos[['idproduto', 'categoria', 'volume', 'massa']]

In [5]:

###CRIANDO TRANSACAO###

dim_transacao = pedidos[['idcliente', 'idvendedor', 'pagamento']]
dim_transacao['sk_cliente'] = range(1, len(dim_transacao) + 1)
dim_transacao = dim_transacao[['sk_cliente', 'idcliente', 'idvendedor', 'pagamento']]

dim_transacao.to_csv(file_path+'silver/dim_transacao.csv')

C:\Users\henri\AppData\Local\Temp\ipykernel_6324\2745140035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_transacao['sk_cliente'] = range(1, len(dim_transacao) + 1)


In [6]:

###CRIANDO ENTREGAS###
entregas = pedidos[['idpedido', 'cepcliente', 'cepvendedor']]
entregas.to_csv(file_path+'silver/dim_entregas.csv')


In [7]:

###CRIANDO PEDIDOS###

# Selecionar e renomear as colunas
fato = pedidos[['idpedido', 'idvendedor', 'idcliente', 'cepvendedor', 'cepcliente', 'idproduto', 'iditem', 'preco', 'frete', 'datacompra', 'datacliente', 'prazo']].copy()

# Converter as colunas de data para o tipo datetime
fato['datacompra'] = pd.to_datetime(fato['datacompra'])
fato['datacliente'] = pd.to_datetime(fato['datacliente'])
fato['prazo'] = pd.to_datetime(fato['prazo'])

# Calcular a diferença em dias para ped_diasentrega e ped_diasprazo
fato['ped_diasentrega'] = (fato['datacliente'] - fato['datacompra']).dt.days
fato['ped_diasprazo'] = (fato['prazo'] - fato['datacompra']).dt.days

final = pd.merge(fato, produtos, on='idproduto', how='left')
final = final[['idpedido', 'idvendedor', 'idcliente', 'idproduto', 'iditem', 'preco', 'frete', 'volume', 'massa', 'ped_diasentrega', 'ped_diasprazo']]

final.to_csv(file_path+'silver/fato_pedidos.csv')